In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
hyperparameters = {
    'num_classes': 9,
    'cnn_channels': [32, 32, 64, 64, 128, 128, 9],
    'lr': 1e-4, # 0.009682778562612496
    'batch_size': 64,
    'val_split': 0.2,
    'test_split': 0.2,
    'num_workers': 4,
    'max_epochs': 50,
}

In [ ]:
import pandas as pd
from dataset import SudokuDataModule

data = pd.read_csv('data/sudoku_small.csv')
dataset = SudokuDataModule(data, hyperparameters)

In [ ]:
from model import SudokuSolverCNN
from lightning.pytorch.utilities.model_summary import ModelSummary

model = SudokuSolverCNN(hyperparameters)
ModelSummary(model, max_depth=2)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.tuner.tuning import Tuner
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor, LearningRateFinder

trainer = Trainer(
    max_epochs=hyperparameters['max_epochs'],
    logger=TensorBoardLogger('logs', name='sudoku-solver', log_graph=True),
    callbacks=[
        ModelCheckpoint(monitor='val_loss', save_top_k=1, mode='min'),
        EarlyStopping(monitor='val_loss', patience=5, mode='min'),
        LearningRateMonitor(logging_interval='step'),
        LearningRateFinder(min_lr=1e-6, max_lr=1e-2, num_training_steps=100000, mode='exponential'),
    ],
    enable_checkpointing=True,
    enable_progress_bar=True,
)

In [ ]:
trainer.fit(model, datamodule=dataset)

In [ ]:
trainer.test(model, datamodule=dataset)

In [ ]:
import torch

puzzle = "020008004090100600003000010901400008000000302005083007004000920700300850000256403"
puzzle = [int(digit) for digit in puzzle]

puzzle_grid = torch.tensor(puzzle, dtype=torch.float32).view(9, 9)
puzzle_grid

In [ ]:
puzzle_one_hot = torch.zeros(9, 9, 9)
for i in range(9):
    for j in range(9):
        if puzzle_grid[i, j] > 0:
            puzzle_one_hot[i, j, int(puzzle_grid[i, j]) - 1] = 1
            
puzzle_one_hot